<h3>Toronto Neighborhoods Assignment</h3><br>
<br>
Load relevant packages

In [50]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import folium
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
#from Ipython.display import HTML
from sklearn.cluster import KMeans

ModuleNotFoundError: No module named 'Ipython'

Scrape Wikipedia page and create dataframe

In [2]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(url.content, 'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table), header = 0)[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Remove 'Not assigned' values from Boroughs <br>
Assign Neighborhood Borough value if 'Not assigned'

In [3]:
df_edit = df[df.Borough != 'Not assigned']
df_edit['Neighborhood'] = np.where(df_edit['Neighbourhood'] == 'Not assigned', df_edit['Borough'], df_edit['Neighbourhood'])
df_edit.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Postcode,Borough,Neighbourhood,Neighborhood
2,M3A,North York,Parkwoods,Parkwoods
3,M4A,North York,Victoria Village,Victoria Village
4,M5A,Downtown Toronto,Harbourfront,Harbourfront
5,M5A,Downtown Toronto,Regent Park,Regent Park
6,M6A,North York,Lawrence Heights,Lawrence Heights


Drop 'Neighbourhood' column <br>
Join rows on 'Postcode' and 'Borough' resulting in a series <br>
Make series into dataframe and index it

In [4]:
df_edit.drop(['Neighbourhood'], inplace = True, axis =1)
df_fin = df_edit.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(','. join)
df_fin = pd.DataFrame(df_fin)
df_fin.reset_index(inplace = True)
df_fin.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Tried to use geocoder script to get lattitude and longitude for the neighborhoods, but it didn't work for me.  So I downloaded the csv from Coursera and 'joined' the coordinates

In [5]:
LL_dat = pd.read_csv('http://cocl.us/Geospatial_data')
LL_dat.rename(columns ={'Postal Code': 'Postcode'}, inplace = True)
LL_dat.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now join LL_dat and df_fin on the postal codes and add on latttitude and longitude to df_fin

In [6]:
df_fin_LL = df_fin.merge(LL_dat, on = 'Postcode')
df_fin_LL.sort_values(by = ['Borough'], inplace = True)
df_fin_LL.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
63,M5N,Central Toronto,Roselawn,43.711695,-79.416936
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
64,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307
65,M5R,Central Toronto,"The Annex,North Midtown,Yorkville",43.672710,-79.405678


Let's see a list of all the boroughs and the number of instances 

In [7]:
df_fin_LL.Borough.value_counts()

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
York                 5
East York            5
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

Let's pull out all instances with Toronto in the name. 

In [8]:
selection = ['Downtown Toronto', 'Central Toronto', 'West Toronto', 'East Toronto']
df_select = df_fin_LL.loc[df_fin_LL['Borough'].isin(selection)]
df_select.reset_index(inplace = True)
df_select

,index,Postcode,Borough,Neighborhood,Latitude,Longitude
0,47,M4S,Central Toronto,Davisville,43.704324,-79.388790
1,63,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,64,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307
4,65,M5R,Central Toronto,"The Annex,North Midtown,Yorkville",43.672710,-79.405678
5,48,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
6,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
7,45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
8,49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049
9,50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529


Before rendering a map, let's find the center coordinates for df_select

In [9]:
latitude = df_select['Latitude'].mean()
longitude = df_select['Longitude'].mean()
print("The center of the map should be :", latitude, longitude)

The center of the map should be : 43.667262184210514 -79.38988323421053


In [10]:
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=11)

for idx, row in df_select.iterrows():
    latit = row['Latitude']
    longi = row['Longitude']
    post = row['Neighborhood']
    folium.Marker([latit, longi]).add_to(map_toronto)

map_toronto.save('map.html')

from IPython.display import HTML
HTML('<iframe src=map.html width=700 height=450></iframe>')

In [11]:
CLIENT_ID = 'NL0K2R3LGLIN14JHKXKBBWDHGMENEEQVH5JIQEBW22JSC2WM'
CLIENT_SECRET = '54RCY0GL01N5AQAX4YYS0Q3DG2VWXFRG4PTBRQ25COPMJEDX'
VERSION = '20180605'
LIMIT = 30

print('CLIENT_ID:' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

CLIENT_ID:NL0K2R3LGLIN14JHKXKBBWDHGMENEEQVH5JIQEBW22JSC2WM
CLIENT_SECRET:54RCY0GL01N5AQAX4YYS0Q3DG2VWXFRG4PTBRQ25COPMJEDX


In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,100)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        #print(results)
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name'])for v in results])
            
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                             'Neighborhood_Latitude',
                             'Neighborhood_Longitude',
                             'Venue',
                             'Venue_Latitude',
                             'Venue_Longitude',
                             'Venue_Category']
    return(nearby_venues)

In [14]:
toronto_venue = getNearbyVenues(names = df_select['Neighborhood'],latitudes = df_select['Latitude'],longitudes = df_select['Longitude'])

Davisville
Roselawn
North Toronto West
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Moore Park,Summerhill East
Lawrence Park
Davisville North
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
St. James Town
Cabbagetown,St. James Town
Chinatown,Grange Park,Kensington Market
Harbord,University of Toronto
Christie
Harbourfront East,Toronto Islands,Union Station
Stn A PO Boxes 25 The Esplanade
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
First Canadian Place,Underground city
Business Reply Mail Processing Centre 969 Eastern
Studio District
The Beaches West,India Bazaar
The Danforth West,Riverdale
The Beaches
Parkdale,Roncesvalles
High Park,The Junction South
Brockton,Exhibition Place,Parkdale Village

In [15]:
print(toronto_venue.shape)
toronto_venue.head()

(1721, 7)


,Neighborhood,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Davisville,43.704324,-79.38879,Thobors Boulangerie Patisserie Café,43.704514,-79.388616,Café
1,Davisville,43.704324,-79.38879,Jules Cafe Patisserie,43.704138,-79.388413,Dessert Shop
2,Davisville,43.704324,-79.38879,XO Gelato,43.705177,-79.388793,Dessert Shop
3,Davisville,43.704324,-79.38879,Marigold Indian Bistro,43.702881,-79.388008,Indian Restaurant
4,Davisville,43.704324,-79.38879,Viva Napoli,43.705752,-79.389125,Pizza Place


In [16]:
toronto_venue.groupby('Neighborhood').count()

,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"Brockton,Exhibition Place,Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",15,15,15,15,15,15
"Cabbagetown,St. James Town",50,50,50,50,50,50
Central Bay Street,86,86,86,86,86,86
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


In [17]:
print('There are {} unique categories.'.format(len(toronto_venue['Venue_Category'].unique())))

There are 236 unique categories.


In [18]:
#onehot encoding
toronto_onehot = pd.get_dummies(toronto_venue[['Venue_Category']], prefix="", prefix_sep="")

#add neighborhood to dataframe
toronto_onehot['Neighborhood'] = toronto_venue['Neighborhood']

#reorder columns -- Neighborhood first
colm = list(toronto_onehot)
colm.insert(0, colm.pop(colm.index('Neighborhood')))
toronto_onehot = toronto_onehot.loc[:, colm]

toronto_onehot.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Davisville,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Davisville,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Davisville,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Davisville,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Davisville,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
toronto_grpd = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grpd.head(10)

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,...,0.00,0.0,0.0,0.010000,0.000000,0.000000,0.010000,0.0,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00,0.0,0.0,0.017544,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.045455
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.052632
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,0.000000,...,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
5,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.020000,...,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.011628,...,0.00,0.0,0.0,0.011628,0.000000,0.000000,0.011628,0.0,0.000000,0.011628
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.01,0.0,0.0,0.060000,0.000000,0.050000,0.010000,0.0,0.000000,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
9,Church and Wellesley,0.011236,0.011236,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.011236,...,0.00,0.0,0.0,0.000000,0.011236,0.011236,0.000000,0.0,0.011236,0.022472


Top 5 venues for each neighborhood

In [20]:
top_venues = 5

for hood in toronto_grpd['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grpd[toronto_grpd['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3      Thai Restaurant  0.04
4  American Restaurant  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2    Steakhouse  0.04
3    Restaurant  0.04
4   Cheese Shop  0.04


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0  Breakfast Spot  0.09
1            Café  0.09
2     Coffee Shop  0.09
3     Yoga Studio  0.05
4             Bar  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.05
2       Auto Workshop  0.05
3                Park  0.05
4          Comic Shop  0.05


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.20
1    Airport Lounge  0.13
2  Airport Terminal  0.13
3

In [21]:
def return_most_common_venues(row, top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:top_venues]

Top 10 venues for each neighborhood

In [22]:
top_venues = 10

indicators = ['st','nd','rd']

#create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
#create new dataframe
neigh_venues_sort = pd.DataFrame(columns = columns)
neigh_venues_sort['Neighborhood']= toronto_grpd['Neighborhood']

for ind in np.arange(toronto_grpd.shape[0]):
    neigh_venues_sort.iloc[ind, 1:] = return_most_common_venues(toronto_grpd.iloc[ind,:], top_venues)
    
neigh_venues_sort.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Restaurant,Sushi Restaurant,Asian Restaurant,Bakery,Bar
1,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Steakhouse,Seafood Restaurant,Farmers Market,Café,Pub,Restaurant
2,"Brockton,Exhibition Place,Parkdale Village",Breakfast Spot,Café,Coffee Shop,Yoga Studio,Bar,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym,Grocery Store
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Park,Smoke Shop,Skate Park,Brewery,Burrito Place,Restaurant,Recording Studio,Comic Shop
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Terminal,Airport Lounge,Harbor / Marina,Plane,Sculpture Garden,Boutique,Boat or Ferry,Airport Gate,Airport Food Court


Cluster neighborhoods

In [30]:
# set of clusters
klusters = 4

toronto_grpd_clustering = toronto_grpd.drop('Neighborhood',1)

toronto_grpd_clustering.head()

# run kmeans
kmeans = KMeans(n_clusters=klusters, random_state=0).fit(toronto_grpd_clustering)

# chk cluster label
kmeans.labels_[0:10]


array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [39]:
neigh_venues_sort.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merge = df_select

toronto_merge = toronto_merge.join(neigh_venues_sort.set_index('Neighborhood'), on='Neighborhood')

toronto_merge.head()

,index,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,47,M4S,Central Toronto,Davisville,43.704324,-79.388790,3,Sandwich Place,Dessert Shop,Pizza Place,Sushi Restaurant,Italian Restaurant,Café,Coffee Shop,Park,Seafood Restaurant,Burger Joint
1,63,M5N,Central Toronto,Roselawn,43.711695,-79.416936,1,Garden,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
2,46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,3,Clothing Store,Sporting Goods Shop,Coffee Shop,Spa,Sandwich Place,Salon / Barbershop,Rental Car Location,Chinese Restaurant,Yoga Studio,Bagel Shop
3,64,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307,0,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Dim Sum Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
4,65,M5R,Central Toronto,"The Annex,North Midtown,Yorkville",43.672710,-79.405678,3,Coffee Shop,Café,Sandwich Place,Pizza Place,Indian Restaurant,Pharmacy,Cosmetics Shop,Pub,Cheese Shop,Burger Joint


Visualize clusters

In [57]:

#create map
map_clusters = folium.Map(location = [latitude, longitude], zoom_start=11)

#set color scheme for clusters
x = np.arange(klusters)
ys = [i + x + (i*x)**2 for i in range(klusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#add markers to map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merge['Latitude'], toronto_merge['Longitude'], toronto_merge['Neighborhood'], toronto_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + 'Cluster' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius = 5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters.save('map3.html')

#from IPython.display import HTML
HTML('<iframe src=map3.html width=700, height=450></iframe>')

In [60]:
toronto_merge.loc[toronto_merge['Cluster Labels']==0, toronto_merge.columns[[1]+list(range(5, toronto_merge.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M5P,-79.411307,0,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Dim Sum Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
6,M4N,-79.388790,0,Park,Swim School,Bus Line,Yoga Studio,Dim Sum Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
9,M4W,-79.377529,0,Park,Playground,Trail,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [61]:
toronto_merge.loc[toronto_merge['Cluster Labels']==1, toronto_merge.columns[[1]+list(range(5, toronto_merge.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M5N,-79.416936,1,Garden,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [62]:
toronto_merge.loc[toronto_merge['Cluster Labels']==2, toronto_merge.columns[[1]+list(range(5, toronto_merge.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,M4T,-79.38316,2,Playground,Gym,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [63]:
toronto_merge.loc[toronto_merge['Cluster Labels']==3, toronto_merge.columns[[1]+list(range(5, toronto_merge.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4S,-79.388790,3,Sandwich Place,Dessert Shop,Pizza Place,Sushi Restaurant,Italian Restaurant,Café,Coffee Shop,Park,Seafood Restaurant,Burger Joint
2,M4R,-79.405678,3,Clothing Store,Sporting Goods Shop,Coffee Shop,Spa,Sandwich Place,Salon / Barbershop,Rental Car Location,Chinese Restaurant,Yoga Studio,Bagel Shop
4,M5R,-79.405678,3,Coffee Shop,Café,Sandwich Place,Pizza Place,Indian Restaurant,Pharmacy,Cosmetics Shop,Pub,Cheese Shop,Burger Joint
7,M4P,-79.390197,3,Gym,Clothing Store,Breakfast Spot,Burger Joint,Food & Drink Shop,Sandwich Place,Hotel,Asian Restaurant,Park,Creperie
8,M4V,-79.400049,3,Coffee Shop,Pub,Sushi Restaurant,Bagel Shop,Fried Chicken Joint,Sports Bar,American Restaurant,Convenience Store,Vietnamese Restaurant,Pizza Place
10,M4Y,-79.383160,3,Japanese Restaurant,Coffee Shop,Gay Bar,Sushi Restaurant,Burger Joint,Restaurant,Gym,Fast Food Restaurant,Pub,Men's Store
11,M5A,-79.360636,3,Coffee Shop,Café,Park,Pub,Bakery,Mexican Restaurant,Restaurant,Theater,Breakfast Spot,Brewery
12,M5B,-79.378937,3,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Thai Restaurant,Pizza Place,Sporting Goods Shop,Hotel,Italian Restaurant
13,M5E,-79.373306,3,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Steakhouse,Seafood Restaurant,Farmers Market,Café,Pub,Restaurant
14,M5G,-79.387383,3,Coffee Shop,Italian Restaurant,Bar,Bubble Tea Shop,Burger Joint,Café,Ice Cream Shop,Sushi Restaurant,Japanese Restaurant,Spa
